**MLP with IMAGE CLASSIFICATION**

In [1]:
import copy 
import random
import time
import glob
import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
import matplotlib.pyplot as plt


**Now we created Dataset.**

3 classes : 
          Drone
          Airplane
          Helicopter

In [2]:
#Oluşturduğumuz dataseti drive'a yükledik. Drive'dan google colab'e çekmek için kullanmamız gereken

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#yüklediğimiz zip dosya uzantılı dataseti zip'ten çıkarmak için 
!unzip -q /content/drive/MyDrive/FIZ437E/Sky-Dataset.zip -d ../

In [9]:
path_dataset = '/content/drive/MyDrive/FIZ437E/sky-Dataset'

In [12]:
mean_ = torch.tensor([0.490, 0.460, 0.40], dtype=torch.float32)
std_ = torch.tensor([0.230, 0.220, 0.223], dtype=torch.float32)

transform = transforms.Compose([transforms.RandomRotation(5),
                                transforms.Resize(256),
                                transforms.RandomCrop(256, padding=5),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean_, std=std_)
])

DATASET = torchvision.datasets.ImageFolder(path_dataset, 
                                           transform=transform)

#datasetimizin %85'i train %15 i validation olsun.
len_dataset = len(DATASET)
len_train = round(len(DATASET)*(0.85))
len_validation = round(len(DATASET)*(0.15))

print('Number of all examples: ',len_dataset)
print('Number of training examples: ',len_train)
print('Number of testing examples: ',len_validation)

Number of all examples:  5599
Number of training examples:  4759
Number of testing examples:  840


In [13]:
train_dataset, test_dataset = torch.utils.data.random_split(DATASET, (len_train, len_validation))

train_data = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=24,     # (number of classes + 5 )* 3
                                              shuffle=True,
)

test_data = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=24,
                                              shuffle=False,
)

**Defining MLP Model**

In [14]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
                
        self.input_fc = nn.Linear(input_dim, 15)
        self.hidden_fc = nn.Linear(15, 12)
        self.output_fc = nn.Linear(12, output_dim)
        
    def forward(self, x):
        
        #x = [batch size, height, width]
        
        batch_size = x.shape[0]

        x = x.view(batch_size, -1)
        
        #x = [batch size, height * width]
        
        h_1 = F.relu(self.input_fc(x))
        
        #h_1 = [batch size, 250]

        h_2 = F.relu(self.hidden_fc(h_1))

        #h_2 = [batch size, 100]

        y_pred = self.output_fc(h_2)
        
        #y_pred = [batch size, output dim]
        
        return y_pred, h_2

In [15]:
input_dimension = 256 * 256* 3
output_dimension = 10

model = MLP(input_dimension, output_dimension)

Optimization  and Loss Function

In [16]:
optimizer = optim.Adam(model.parameters())
#create cross entropy
criterion = nn.CrossEntropyLoss()

def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc


Train Function

In [25]:
def train(model, iterator, optimizer, criterion,epoch):
    
    epoch_loss_train = 0
    epoch_accuracy_train = 0
    
    model.train()
    
    for i,(x, y) in enumerate(iterator):
      
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss_train += loss.item()
        epoch_accuracy_train += acc.item()

        if i%40 == 0:
            print('Epoch: [{}]/({}/{}), Train Loss: {:.4f}, Accuracy: {:.2f}'.format(
                epoch, i,len(train_data),loss.item(), epoch_accuracy_train / len(iterator)))
        
    return epoch_loss_train / len(iterator), epoch_accuracy_train / len(iterator)

Test Function

In [26]:
def evaluate(model, iterator, criterion):
    epoch_loss_test = 0
    epoch_accuracy_test = 0
      
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss_test += loss.item()
            epoch_accuracy_test += acc.item()

    print('Epoch: [{}], Test Loss: {:.4f}, Accuracy: {:.2f}'.format(
    epoch, epoch_loss_test / len(iterator), epoch_accuracy_test / len(iterator)))
        
    return epoch_loss_test / len(iterator), epoch_accuracy_test / len(iterator)

**Training the Model**

In [19]:
Epochs = 30

best_valid_loss = float('inf')

for epoch in range(Epochs):   
    
    train_loss, train_acc = train(model, train_data, optimizer, criterion,epoch)
    valid_loss, valid_acc = evaluate(model, test_data, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')


Epoch: [0]/(0/199), Train Loss: 2.2135, Accuracy: 0.00
Epoch: [0]/(40/199), Train Loss: 2.3549, Accuracy: 0.16
Epoch: [0]/(80/199), Train Loss: 1.4119, Accuracy: 0.32
Epoch: [0]/(120/199), Train Loss: 0.2563, Accuracy: 0.49
Epoch: [0]/(160/199), Train Loss: 0.7786, Accuracy: 0.65
Epoch: [0], Test Loss: 0.5696, Accuracy: 0.86
Epoch: [1]/(0/199), Train Loss: 0.4373, Accuracy: 0.00
Epoch: [1]/(40/199), Train Loss: 0.3739, Accuracy: 0.18
Epoch: [1]/(80/199), Train Loss: 0.4677, Accuracy: 0.35
Epoch: [1]/(120/199), Train Loss: 0.3429, Accuracy: 0.51
Epoch: [1]/(160/199), Train Loss: 0.0664, Accuracy: 0.69
Epoch: [1], Test Loss: 0.5442, Accuracy: 0.87
Epoch: [2]/(0/199), Train Loss: 3.0562, Accuracy: 0.00
Epoch: [2]/(40/199), Train Loss: 0.3336, Accuracy: 0.17
Epoch: [2]/(80/199), Train Loss: 0.8463, Accuracy: 0.35
Epoch: [2]/(120/199), Train Loss: 0.1551, Accuracy: 0.52
Epoch: [2]/(160/199), Train Loss: 0.7276, Accuracy: 0.69
Epoch: [2], Test Loss: 0.6343, Accuracy: 0.87
Epoch: [3]/(0/199),

"\n    print(f'Epoch: {epoch+1:02}')\n    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')\n    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')\n    "

Loss of MLP Model :  0.3466

Accuracy of MLP Model : %90